In [7]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]
    context: dict


graph_builder = StateGraph(State)

import os
from langchain_openai import AzureChatOpenAI


os.environ["AZURE_OPENAI_API_KEY"] = "2RT8mQ01jQmMrZwiHczy75cckYTP2Kii9McSvgDCGFJ8Q7P37vVwJQQJ99BJACYeBjFXJ3w3AAAAACOGxEp9"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://chr-umn-hackathon-resource.cognitiveservices.azure.com/"
os.environ["OPENAI_API_VERSION"] = "2024-12-01-preview"

llm = AzureChatOpenAI(
    azure_deployment= "gpt-5-mini",
    api_version = "2024-12-01-preview",
    azure_endpoint = "https://chr-umn-hackathon-resource.cognitiveservices.azure.com/",
    api_key = "2RT8mQ01jQmMrZwiHczy75cckYTP2Kii9McSvgDCGFJ8Q7P37vVwJQQJ99BJACYeBjFXJ3w3AAAAACOGxEp9",
)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

#Entry point
graph_builder.add_edge(START, "chatbot")

#Exit point
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

#Visualizer
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [9]:
example_loads = {
    "Atlanta-Chicago": {
        "origin": "Atlanta, GA",
        "destination": "Chicago, IL",
        "commodity": "Frozen poultry (reefer required)",
        "weight": "38,000 lbs",
        "distance": "~720 miles",
        "target_price": 1350
    },
    "Dallas-LA": {
        "origin": "Dallas, TX",
        "destination": "Los Angeles, CA",
        "commodity": "Electronics (dry van)",
        "weight": "30,000 lbs",
        "distance": "~1,450 miles",
        "target_price": 2250
    },
    "Seattle-Denver": {
        "origin": "Seattle, WA",
        "destination": "Denver, CO",
        "commodity": "Apples (reefer, temp-controlled 34°F)",
        "weight": "42,000 lbs",
        "distance": "~1,300 miles",
        "target_price": 2000
    },
    "Memphis-Miami": {
        "origin": "Memphis, TN",
        "destination": "Miami, FL",
        "commodity": "Consumer packaged goods (dry van)",
        "weight": "44,000 lbs",
        "distance": "~1,050 miles",
        "target_price": 1700
    },
    "Newark-Boston": {
        "origin": "Newark, NJ",
        "destination": "Boston, MA",
        "commodity": "Pharmaceuticals (reefer, high value)",
        "weight": "25,000 lbs",
        "distance": "~260 miles",
        "target_price": 850
    }
}


In [11]:
# def stream_graph_updates(user_input: str, route: str):
#     if route not in example_loads:
#         print("Unknown route. Available routes:", list(example_loads.keys()))
#         return

#     load = example_loads[route]

#     system_prompt = {
#         "role": "system",
#         "content": (
#             "You are a professional freight negotiation assistant. "
#             "You help logistics coordinators negotiate fair rates for hauling loads. "
#             "Be realistic, polite, and strategic — aim to meet or beat the target price, "
#             "but justify your reasoning based on distance, commodity type, and market trends."
#         )
#     }

#     context_message = {
#         "role": "system",
#         "content": (
#             f"You are negotiating for this load:\n"
#             f"Origin: {load['origin']}\n"
#             f"Destination: {load['destination']}\n"
#             f"Commodity: {load['commodity']}\n"
#             f"Weight: {load['weight']}\n"
#             f"Distance: {load['distance']}\n"
#             f"Target Price: ${load['target_price']}"
#         )
#     }

#     message_sequence = [
#         system_prompt,
#         context_message,
#         {"role": "user", "content": user_input}
#     ]


#     for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
#         for value in event.values():
#             print("Assistant:", value["messages"][-1].content)


# route = input("Select a route (e.g., Atlanta-Chicago): ")
# while True:
#     try:
#         user_input = input("You: ")
#         if user_input.lower() in ["quit", "exit", "q"]:
#             print("Goodbye!")
#             break
#         stream_graph_updates(user_input)
#     # except:
#     #     # fallback if input() is not available
#     #     # user_input = "What do you know about LangGraph?"
#     #     # print("User: " + user_input)
#     #     # stream_graph_updates(user_input)
#     #     # break
#     except Exception as e:
#         print("Error:", e)
#         break
def stream_graph_updates(user_input: str, route: str):
    if route not in example_loads:
        print("Unknown route. Available routes:", list(example_loads.keys()))
        return

    load = example_loads[route]

    system_prompt = {
        "role": "system",
        "content": (
            "You are a professional freight negotiation assistant. "
            "You help logistics coordinators negotiate fair rates for hauling loads. "
            "Be realistic, polite, and strategic — aim to meet or beat the target price, "
            "but justify your reasoning based on distance, commodity type, and market trends."
        )
    }

    context_message = {
        "role": "system",
        "content": (
            f"You are negotiating for this load:\n"
            f"Origin: {load['origin']}\n"
            f"Destination: {load['destination']}\n"
            f"Commodity: {load['commodity']}\n"
            f"Weight: {load['weight']}\n"
            f"Distance: {load['distance']}\n"
            f"Target Price: ${load['target_price']}"
        )
    }

    message_sequence = [
        system_prompt,
        context_message,
        {"role": "user", "content": user_input}
    ]

    for event in graph.stream({"messages": message_sequence}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


# 主循环
route = input("Select a route (e.g., Atlanta-Chicago): ")

while True:
    try:
        user_input = input("You: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        # ✅ 改成两个参数
        stream_graph_updates(user_input, route)
    except Exception as e:
        print("Error:", e)
        break


Assistant: Hey — thanks for reaching out. Before I start outreach, a couple quick clarifying questions so I can negotiate confidently for you:

- Exact pickup and delivery dates/windows? Any appointment times at origin or destination?
- Pallet count and dimensions (palletized, stacked, floor-loaded)?
- Required reefer temperature and pre-cool requirement?
- Do you offer quick pay (e.g., 48-hour) or any accessorials you’re willing to cover (detention, layover, tarping, unloading)?
- Is the load dock-to-dock or will driver need a liftgate/driver assistance?

Assuming standard conditions (53’ reefer, dock-to-dock, no appointments beyond normal hours, 0°F setpoint, typical 2‑hour free layover), here’s a concise negotiation plan to get the load moved at or below your $1,350 target for the ~720 mile haul (reefer FTL, 38,000 lbs):

Market context (short):
- 720 miles with a full-ish reefer load is roughly $1.50–$2.00/mile in many markets. $1,350 equates to about $1.88/mile — a reasonable, com